Ensemble 관련해서 찾아본 결과, 먼저 train data를 여러 개로 나눠서 각각에 대한 모델을 학습한 뒤 
그 결과를 이용해 새로운 classifier를 한 번 더 학습하는 Stacking이라는 방법이 있는데 
이는 train data가 부족한 저희 경우에는 맞지 않는 것 같았고, 다른 관련 논문들도 대부분 다시 새로운 모델을 학습해야 하는 케이스라
저희 조원 각각이 구한 submission file을 최적 가중치를 통해 ensemble하는 방법을 찾아보았습니다.

아래 코드는 제가 이전에 구했던 3개의 submission file을 ensemble해서 최적의 weight를 구하기 위한 것입니다.
train해 두었던 모델 3개를 각각 다시 불러와 validation set을 통해 predicted probability를 각각 구한 뒤,
여기에 w1+w2+w3=1인 weights를 이용해 가중평균을 낸 final_preditions을 바탕으로 weight를 update시켜가며 
최적 해를 찾아보려 했습니다.(scipy.optimize.minimize() 이용)
하지만 validation set이 test set에 비해 너무 작고, metric이 roc_auc여서 최적화 과정에서 직접 loss function으로 사용할 수가 없어서 그런지
ensemble한 성적이 그닥 좋지는 않았습니다.
또한 제 경우는 초기 weight 값들이 조금만 달라져도 전혀 다른 지점으로 수렴을 하는 문제 역시 발생했습니다...

혹시 가지고 있는 여러 submission file ensemble 해 보시고 싶은 분은 아래 코드 써 보세요!
(다만 이전에 학습했던 모델을 다시 불러와서 validation set에 대한 예측 확률을 다시 구해야 weight를 구할 수 있어서 조금 귀찮을 수도...ㅠ)

In [157]:
test_cleaned = pd.read_csv('preprocessed_test.csv')
idlist = test_cleaned['id']

In [43]:
from tensorflow.contrib import learn
import numpy as np

vocab_processor = learn.preprocessing.VocabularyProcessor(500)
X = np.array(list(vocab_processor.fit_transform(train_cleaned_7['comment_text'].astype(str))))

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_valid = train_test_split(X, test_size=0.05, random_state=1234)
Y_train, Y_valid = train_test_split(train_target_7, test_size=0.05, random_state=1234)

In [106]:
Y_valid=Y_valid.drop('all_zero', axis=1)
Y_valid.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
12345,0,0,0,0,0,0
55587,0,0,0,0,0,0
16877,0,0,0,0,0,0
140039,0,0,0,0,0,0
125777,0,0,0,0,0,0


In [141]:
Y_valid_values = Y_valid.values

In [45]:
cols_target_7 = ['toxic','severe_toxic','obscene','threat','insult','identity_hate', 'all_zero']
train_target_7 = train_cleaned_7[cols_target_7]
train_target_7.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,all_zero
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [47]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer 
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W ,b, name = "scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
        
        with tf.name_scope("prob_each"):
            self.prob_each = tf.nn.sigmoid(self.scores, name="prob_each")
       
        with tf.name_scope("loss"):
            #losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            losses = tf.nn.weighted_cross_entropy_with_logits(targets=self.input_y, logits=self.scores, pos_weight=tf.constant([1,10,2,30,2,10,0.1]))    
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    

In [48]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [49]:

#import tensorflow as tf
#import numpy as np
import os
import time
import datetime
#from tensorflow.contrib import learn

# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 25, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.5, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 59, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 3, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

tf.logging.set_verbosity(tf.logging.INFO)


# Data Preparation
# ==================================================

# Load data
#print("Loading data...")
#x_text, y = #data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

# Build vocabulary
#max_document_length = max([len(x.split(" ")) for x in x_text])
#vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
#x = np.array(list(vocab_processor.fit_transform(x_text)))

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(Y_train_7)))
x_shuffled = X[shuffle_indices]
y_shuffled = Y_train_7[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(Y_train_7)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

#del x, y, x_shuffled, y_shuffled

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=59
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=25
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.5
LOG_DEVICE_PLACEMENT=False
NUM_CHECKPOINTS=5
NUM_EPOCHS=3
NUM_FILTERS=128

Vocabulary Size: 199560
Train/Dev split: 143614/15957


In [72]:
result_7_1=[]
result_7_2=[]
result_7_3=[]

In [89]:
#latest model

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=X.shape[1], 
            num_classes=Y_train_7.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3) 
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

       
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)        
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
        scores_summary = tf.summary.tensor_summary("scores", cnn.scores)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary]) #, grad_summaries_merged
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        
        # Restore trained model
        new_saver = tf.train.import_meta_graph('./runs/1517809492/checkpoints/model-13500.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint('./runs/1517809492/checkpoints/'))
        
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy, scores, prob_each= sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy, cnn.scores, cnn.prob_each],
                feed_dict)
            
            if step % 100 == 0:
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}".format(time_str, step, loss))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy, scores = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy, cnn.scores],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}".format(time_str, step, loss))
            if writer:
                writer.add_summary(summaries, step)
                
        def test_step(x_batch):
           
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.dropout_keep_prob: 1.0
            }
            prob_each = sess.run(cnn.prob_each,feed_dict)
            result_7_3.append(prob_each)
            
        print("Model is restored.")
        
        batches_test = batch_iter(X_valid, FLAGS.batch_size, num_epochs=1, shuffle=False)
        for batch in batches_test:
            test_step(batch)
       
        print("The End!")

Writing to C:\Users\User\Desktop\SNU\Deepest\Toxic Comment Classification\runs\1518955080

INFO:tensorflow:Restoring parameters from C:\Users\User\Desktop\SNU\Deepest\Toxic Comment Classification\runs\1517809492\checkpoints\model-13500
Model is restored.
The End!


In [91]:
import copy

c1 = copy.deepcopy(result_7_3) #1517809492
c2 = copy.deepcopy(result_7_2) #1518157525
c3 = copy.deepcopy(result_7_1) #1518171722, len=135*59+14

In [93]:
df1 = pd.concat([pd.DataFrame(c1[i], columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate','all_zero']) for i in range(len(c1))], ignore_index=True)
df2 = pd.concat([pd.DataFrame(c2[i], columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate','all_zero']) for i in range(len(c2))], ignore_index=True)
df3 = pd.concat([pd.DataFrame(c3[i], columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate','all_zero']) for i in range(len(c3))], ignore_index=True)

In [95]:
df1 = df1.drop('all_zero', axis=1)
df2 = df2.drop('all_zero', axis=1)
df3 = df3.drop('all_zero', axis=1)

In [96]:
arr1=df1.values
arr2=df2.values
arr3=df3.values

In [97]:
arr1_train, arr1_test = train_test_split(arr1, test_size=0.05, random_state=1234)
arr2_train, arr2_test = train_test_split(arr2, test_size=0.05, random_state=1234)
arr3_train, arr3_test = train_test_split(arr3, test_size=0.05, random_state=1234)

In [107]:
predictions = []
predictions.append(arr1)
predictions.append(arr2)
predictions.append(arr3)

In [148]:
import math
from scipy.optimize import minimize

# 원래 참고한 코드에서는 log loss 였는데 multi-class에 맞게 수정했습니다.
def loss_func(weights):
    final_prediction = np.zeros_like(arr1)
    for weight, prediction in zip(weights, predictions):
            final_prediction += weight * prediction
    loss = 0
    for i in range(len(final_prediction)):
        for j in range(len(final_prediction[0])):
            loss += np.sum(final_prediction[i][j] - np.multiply(final_prediction[i][j], Y_valid_values[i][j]) + math.log(1 + np.exp(-final_prediction[i][j])))
    return loss

In [ ]:
#its better to choose many random starting points and run minimize a few times
#시작점을 조금씩만 바꿔도 최적 weight가 많이 바뀌어서 자꾸 local minimum에 빠집니다...
#metric이 roc_auc이기도 하고, validation set이 test set에 비해 매우 작은 편이라 그런지 
#여기서 구한 최적 weight로 구한 결과로도 실제 score는 잘 안 나오네요ㅠ

starting_values = [0.333, 0.333, 0.334]

cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
#our weights are bound between 0 and 1
bounds = [(0,1)]*len(predictions)

res = minimize(loss_func, starting_values, method='SLSQP', bounds=bounds, constraints=cons)

print('Best Weights: {weights}'.format(weights=res['x']))

In [161]:
df1_test = pd.read_csv("submission_1_7.csv")
df2_test = pd.read_csv("submission_2_7.csv")
df3_test = pd.read_csv("submission_3_7.csv")

In [162]:
df1_test=df1_test.drop('id', axis=1)
df2_test=df2_test.drop('id', axis=1)
df3_test=df3_test.drop('id', axis=1)

In [163]:
arr1_test=df1_test.values
arr2_test=df2_test.values
arr3_test=df3_test.values

In [164]:
cols_target = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
a = 0.5 * arr1_test + 0.5 * arr3_test
df_a=pd.DataFrame(a, columns=cols_target)
df_a.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.942185,0.031438,0.777922,0.053484,0.611630,0.066683
1,0.002528,0.000283,0.002986,0.000126,0.003983,0.000878
2,0.050209,0.019454,0.041816,0.027878,0.050728,0.029217
3,0.001057,0.000316,0.001267,0.000110,0.001885,0.000310
4,0.006905,0.001157,0.006825,0.000819,0.009312,0.002638


In [169]:
df_a['id']=idlist
df_a=df_a[['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
df_a.to_csv('optimization_1.csv', index=False)

In [171]:
b = 0.5 * arr1_test + 0.5 * arr2_test
df_b=pd.DataFrame(b, columns=cols_target)
df_b['id']=idlist
df_b=df_b[['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
df_b.to_csv('optimization_2.csv', index=False)